In [ ]:
from ratdata import data_manager as dm, process, ingest
from pathlib import Path
import numpy as np
import scipy.signal as signal

In [ ]:
dm.db_connect('rat_data.db')

In [ ]:
all_recordings = dm.RecordingFile.select().count()
for i, r in enumerate(dm.RecordingFile.select()):
    full_filename = Path(r.dirname) / r.filename
    data = ingest.read_mce_matlab_file(str(full_filename))
    x = np.mean(data.electrode_data, 0)
    fs = int(1/data.dt)
    ff, pxx = signal.welch(x, fs, nperseg=fs)
    m, b = process.fit_oof(ff, pxx, 2, 100)
    if r.stim.count() != 0 and r.stim.get().stim_type != 'nostim':
        stim = True
    else:
        stim = False
    print('[%d/%d] %s %.2f %d' % (i+1, all_recordings, r.filename, m, stim))
        